# Compter des occurrences : l’exemple des mots d’un texte

## De quoi il s’agit ?

Compter des occurrences revient à subdiviser un objet en éléments et à compter le nombre de fois où chaque élément apparaît. Prenons une liste aléatoire de cent nombres entiers entre 0 et 20 et calculons combien de fois apparaît le nombre 9 :

In [ ]:
from random import randint

l = [ randint(0, 20) for n in range(0, 100) ]

print(f"Le nombre 9 apparaît { l.count(9) } fois.")

On peut réaliser une opération similaire en comptabilisant les apparitions de la lettre *e* dans une phrase :

In [ ]:
sent = "En pratique, un pêcheur pêche avec une canne."

print(f"La lettre 'e' apparaît { sent.count('e') } fois.")

Python dissocie par défaut le caractère *e* de ses versions accentuée *ê* et majuscule *E*. Plusieurs stratégies peuvent être mises en place pour y remédier :
- remplacer les caractères ;
- additionner les occurrences de chaque cas ;
- normaliser la phrase.

Prenons la dernière stratégie en faisant appel à la méthode `.normalize()` du module `unicodedata`, qui décompose une lettre en ses différents constituants :

In [ ]:
import unicodedata

def normalize(s):
    """Returns the normalized version of a string.

    s -- string to normalize
    """
    normalized_string = str()
    for c in s:
        components = unicodedata.normalize('NFKD', c)
        base = components[0]
        normalized_string += base.lower()

    return normalized_string

print(normalize(sent))

Grâce à la fonction `map()`, on peut obtenir le même résultat plus rapidement :

In [ ]:
sent = map(lambda x: unicodedata.normalize('NFKD', x)[0], sent)
sent = ''.join(sent)

print(f"La lettre 'e' apparaît { sent.count('e') } fois.")

Et pour basculer en bas de casse :

In [ ]:
sent = sent.lower()

print(f"La lettre 'e' apparaît { sent.count('e') } fois.")

## Préparer les données

La question de compter des occurrences de mots n’est donc pas si anodine qu’elle peut paraître. La qualité du résultat dépend grandement de la définition de l’objectif que l’on se fixe. Parfois, en effet, nous voudrons intégrer la ponctuation, d’autres fois ne compter que des formes verbales, ou supprimer les mots non signifiants etc.

Prenons un énoncé :

> Le petit chat boit de l’eau. Le petit chien boit du lait.

Nous aimerions au final obtenir une liste de tuples de mots triée par leur fréquence d’apparition, puis par ordre alphabétique :
```python
[('boit', 2), ('le', 2), ('petit', 2), ('chat', 1)…]
```

**1e étape :** effectuer une tokenisation avec *spaCy*.

In [ ]:
import spacy

nlp = spacy.blank('fr')
doc = nlp('Le petit chat boit de l’eau. Le petit chien boit du lait.')
tokens = list(filter(lambda t: not t.is_punct, doc))

**2e étape :** basculer les tokens en bas de casse.

In [ ]:
tokens = list(
    map(
        lambda t:t.text.lower(),
        tokens
    )
)

**3e étape :** regrouper les tokens par fréquence d’apparition.

In [ ]:
def get_occurrences(tokens):
    """Builds up a dictionary of words and the count of their
    occurrences.

    tokens -- list of tokens
    """

    occurrences = {}
    for token in tokens:
        occurrences.update({
            token: occurrences.get(token, 0) + 1
        })
    return occurrences

occurrences = get_occurrences(tokens)

**4e étape :** trier le dictionnaire par ordre alphabétique.

In [ ]:
occurrences = sorted(
    occurrences.items(),
    key=lambda x:x[0]
)

**5e étape :** trier la liste de tuples par ordre décroissant de fréquence d’apparition.

In [ ]:
occurrences = sorted(
    occurrences,
    key=lambda x:x[1],
    reverse=True
)

## Améliorer le calcul des occurrences

### `defaultdict`

La structure de données `defaultdict` nous permet d’améliorer la constitution du dictionnaire des occurrences :

In [ ]:
from collections import defaultdict

occurrences = defaultdict(int)

for token in tokens:
    occurrences[token] = occurrences[token] + 1

# sorting
occurrences = sorted(occurrences.items(), key=lambda x:x[0])
occurrences = sorted(occurrences, key=lambda x:x[1], reverse=True)

### `Counter`

Il existe toutefois un autre objet du module `collections` qui est encore plus facilement manipulable pour ce genre d’opérations : `Counter`

In [ ]:
from collections import Counter

occurrences = Counter(tokens)

Il a l’avantage d’embarquer une méthode pour afficher la liste des items les plus fréquents :

In [ ]:
occurrences.most_common(5)